In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

%matplotlib inline


In [ ]:
random_state = 23

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
df = pd.read_csv('data/cell2celltrain.csv')

In [ ]:
df.transpose()

In [ ]:
df.describe().transpose()

# Data Cleaning

IncomeGroup is a numerical field with values 0-9, suggesting income already binned into 10 groups. Already binned, we cannot treat this as continuous but need to treat it as a categorical variable.

Most binary columns contain values "Yes" and "No". The exception is columns Homeownership which is given as "Known" and "Unknown" in the dataset. Both types of binary columns to be mapped to 1 and 0. 



In [ ]:
categorical_columns = ['CreditRating', 'PrizmCode', 'Occupation', 'Homeownership', 'MaritalStatus', 'IncomeGroup', 
                       'ServiceArea']
binary_columns = ['Churn', 'ChildrenInHH', 'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner', 'RVOwner',
                  'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings', 'NonUSTravel', 'OwnsComputer',
                  'HasCreditCard', 'NewCellphoneUser', 'NotNewCellphoneUser', 'OwnsMotorcycle', 
                  'MadeCallToRetentionTeam', 'RetentionOffersAccepted' ]
continuous_columns = []
outlier_candidates = []

replace_with_mean = { 'HandsetPrice' : 'Unknown' }

for b_column in binary_columns: 
    df[b_column] = df[b_column].map({'Yes': 1, 'No': 0})

df['Homeownership'] = df['Homeownership'].map({'Known': 1, 'Unknown': 0})
    
for r_column, nastring in replace_with_mean.items(): 
    temp = df[r_column][df[r_column] != nastring].astype(int)
    df[r_column] = df[r_column].replace(nastring, temp.mean()).astype(int)

def drop_from(categorical_columns, binary_columns, continuous_columns, outlier_candidates, drop):
    categorical_columns = list(set(categorical_columns) - set(drop))
    binary_columns = list(set(binary_columns) - set(drop))
    continuous_columns = list(set(continuous_columns) - set(drop))
    outlier_candidates = list(set(outlier_candidates) - set(drop))
    return categorical_columns, binary_columns, continuous_columns, outlier_candidates

## Handling NaN values

We select all columns that have missing values in the dataset. 

In [ ]:
# get all NaN columns
nan_columns_all =  df.loc[:, df.isna().any()].columns
print(nan_columns_all,)

All the continuous For all continuous variables we replace the missing values by the mean of each column. 
The exception is the percentage change of Minutes and Revenue, where we assume no change from the past period for NaN records. For RetentionOffersAccepted, a binary column, we set NaN to zero. 
Finally, we introduce a dummy category for the records where we don't know the service area. 

In [ ]:
zero_columns = ['PercChangeMinutes', 'PercChangeRevenues', 'RetentionOffersAccepted']

# handle percentage changes differently from other continuous variables
nan_columns = nan_columns_all.drop(zero_columns)
df[nan_columns] = df[nan_columns].fillna(df[nan_columns].mean())

df[zero_columns] = df[zero_columns].fillna(0)

# ServiceArea is the only categorical left with NaN values: 
df['ServiceArea'] = df['ServiceArea'].fillna('UNKNOWN')

print("Number of columns left with NaN records: ",df.isna().any().sum())

## Performance metrics

Define the performance metric for classification. We use a recall-heavy f&beta; score with &beta; = 2 to measure performance of all classifiers. 

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer

scorer = make_scorer(fbeta_score, beta=2)

## Benchmark models

We define two benchmark models that we will test our classifiers against: 

1. A simple kNN classfier with k=2 on the whole dataset
2. A naive, deterministic model which simulates targetting anyone whose 1 or 2 year contract might run out

First, one-hot encode all categorical columns and proceed with the benchmark model on a copy of the data. Split the data into train and test set. 

In [ ]:
df_bm = pd.get_dummies(df, columns=categorical_columns)

In [ ]:
from sklearn.model_selection import train_test_split

X = df_bm.drop(columns='Churn', axis=1)
y = df_bm['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)

Training the kNN classifier and returning its f&beta; score. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf_knn = KNeighborsClassifier(n_neighbors=2)

clf_knn.fit(X_train, y_train)

print('kNN Classifier: ', scorer(clf_knn, X_test, y_test))

Defining and training the naive classifier. Without a data driven models for campaign selection available, sales reps will have to rely on simple heuristics to determine which customers to target with a retention campaign. Here we target anyone who is nearing the end of their 1 or 2 year contracts. The model will predict that any customer who is within 3 months of the end of a 1 or 2 year contract will churn. 

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class NaiveIntuitionClassifier(BaseEstimator, ClassifierMixin):

    def fit(self, X, y):
        return self

    
    def predict(self, X):
        X = check_array(X)
        months = [10,11,12,22,23,24]
        D = np.isin(X_test['MonthsInService'], months)
        return D.astype(int)

    
clf_naive = NaiveIntuitionClassifier()
clf_naive.fit(X_train, y_train)

print('Naive Classifier: ', scorer(clf_naive, X_test, y_test))

## Exploratory Data Analysis

In [ ]:
drop_candidates = ['TruckOwner', 'RVOwner', 'OwnsComputer', 'HasCreditCard', 'NonUSTravel', 'Homeownership', 
                   'MaritalStatus', 'ChildrenInHH', 'NewCellphoneUser', 'NotNewCellphoneUser', 'BuysViaMailOrder', 
                   'RespondsToMailOffers', 'OptOutMailings', 'DirectorAssistedCalls', 'ThreewayCalls', 
                   'CallWaitingCalls', 'CallForwardingCalls']

In [ ]:
corrmat = df[drop_candidates+['Churn']].corr()
f, ax = plt.subplots(figsize=(20,15))
sns.heatmap(corrmat, square=True)

# Looks like we have a visualization bug here that will be fixed with 3.1.2 
# https://gitter.im/matplotlib/matplotlib?at=5d239514f5dd1457424d7b09

It is reasonable that service area has some predictive power regarding churn. There might be bad coverage areas where competitor products are more compelling. 

However, there is a huge number of service areas in the dataset, and encoding them would explode our dimensionality, adding over 700 dimensions, if we would encode service area into features. Let's see if we can reduce or remove service area. 

In [ ]:
df['ServiceArea'].unique().shape

Let's look at service areas and churn behavior in more depth. Filter out very small service areas where we have less than 100 subscribers.

In [ ]:
s = df['ServiceArea'].value_counts()
service_areas = df[df.isin(s.index[s > 100]).values]

order = service_areas['ServiceArea'].value_counts().index

fig, ax = plt.subplots(figsize=(20, 8))
sns.countplot(x='ServiceArea', data=service_areas, hue='Churn', order=order)
plt.ylabel('Subscribers')
plt.show()

fig, ax = plt.subplots(figsize=(20, 8))
sns.barplot(x='ServiceArea', y='Churn', data=service_areas, order=order) 
plt.ylabel('Churn rate')
plt.show()

The more subscribers a service area has, the more stable the churn rate. This aligns with intuitive assessment of the data set. Only where the areas are very small (just above the 100 subscriber threshold analyzed) do we see larger variations in churn rate. These small number of samples are not expected to generalize well however. Service area is not considered worth the explosion in dimensionality, and we drop it. 

In [ ]:
drop_candidates.append('ServiceArea')

A unique customer ID has no predictive power as it cannot generalize to new records. Worse, being a running number, it would likely be misinterpreted as a continuous feature. 

Confirm that customer IDs are unique. Add them to the list of columns to be dropped. 

In [ ]:
df['CustomerID'].unique().shape[0] == df.shape[0]

In [ ]:
corrmat = df[['CustomerID', 'Churn']].corr()
f, ax = plt.subplots(figsize=(5,4))
sns.heatmap(corrmat, square=True)
plt.show()

In [ ]:
drop_candidates.append('CustomerID')

Drop the columns we decided to exclude after initial analysis, and make sure they are not included in further processing of categorical and binary columns. 

In [ ]:
df_dropped = df.drop(drop_candidates, axis=1)

categorical_columns, binary_columns, continuous_columns, outlier_candidates = drop_from(
    categorical_columns, binary_columns, continuous_columns, outlier_candidates, drop_candidates)

### List and visualize continuous variables

In [ ]:
from itertools import chain

continuous_columns = [x for x in df_dropped.columns if x not in chain(binary_columns, categorical_columns) ]
print('Continous columns: ', continuous_columns)

fig = plt.figure(figsize = (20, 50))
j = 0
for c_column in continuous_columns:
    plt.subplot(20, 3, j+1)
    j += 1
    sns.distplot(df_dropped[c_column][df_dropped['Churn'] == 0], color='g', label = 'Remain')
    sns.distplot(df_dropped[c_column][df_dropped['Churn'] == 1], color='r', label = 'Churn')
    plt.legend(loc='best')
    
fig.suptitle('Churn Analysis Continous Variables', fontsize=24)
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

We can see that there is a large number of AgeHH1 and AgeHH2 records with age = 0. Likely this means either information is missing, or there is no second houshold member. We cannot replace age with mean as that would just shift the large spike to the mean and misrepresent non-existant house hold members. 
Instead, we consider dropping the features, as it is not expected to explain churning much anyway (especially AgeHH2).

In [ ]:
corrmat = df_dropped[['AgeHH1', 'AgeHH2', 'Churn']].corr()
f, ax = plt.subplots(figsize=(5,4))
sns.heatmap(corrmat, square=True)
plt.show()

In [ ]:
drop = ['AgeHH1', 'AgeHH2']
drop_candidates.extend(drop)
df_dropped = df_dropped.drop(drop, axis=1)

categorical_columns, binary_columns, continuous_columns, outlier_candidates = drop_from(
    categorical_columns, binary_columns, continuous_columns, outlier_candidates, drop)

According to the dataset description in , a HandsetPrice of 0 means missing data on Handsets. We replace 0 price for handsets with the mean of handset price. This makes sense as you cannot use the cell service without a handset. 

In [ ]:
df_dropped['HandsetPrice'] = df_dropped['HandsetPrice'].replace(0, df_dropped['HandsetPrice'].mean())

df_unknowns = df_dropped

### Outlier removal

Some of the features' distributions show outliers. Examples are CallForwardingCalls, ActiveSubs, or RoamingCalls. We are going to remove the outliers by cutting off values outside of 3* standard deviation, per column. 


In [ ]:
outlier_candidates = ['DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls', 'DroppedCalls', 'BlockedCalls', 
                      'UnansweredCalls', 'CustomerCareCalls', 'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 
                      'InboundCalls', 'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls', 
                      'CallForwardingCalls', 'CallWaitingCalls', 'UniqueSubs', 'ActiveSubs', 'RetentionCalls', 
                      'ReferralsMadeBySubscriber', 'AdjustmentsToCreditRating', 'TotalRecurringCharge', 
                      'MonthlyMinutes', 'MonthlyRevenue' ]
categorical_columns, binary_columns, continuous_columns, outlier_candidates = drop_from(
    categorical_columns, binary_columns, continuous_columns, outlier_candidates, drop_candidates)

fig = plt.figure(figsize = (20, 50))
j = 0
for o_column in outlier_candidates: 
    plt.subplot(20, 2, j+1)
    j += 1
    sns.boxplot(x=df_unknowns[o_column])

fig.suptitle('Churn Analysis Continuous Variables', fontsize=24)
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

### Feature engineering

We simplify some of the continuous, descrete features to binary features, where the vast majority of samples are either 0 or 1. 

In [ ]:
print('Subscribers having made a referral',
      df_unknowns['ReferralsMadeBySubscriber'][df_unknowns['ReferralsMadeBySubscriber'] > 0].count())
print('Subscribers having made more than one referral',
    df_unknowns['ReferralsMadeBySubscriber'][df_unknowns['ReferralsMadeBySubscriber'] > 1].count())
print('Subscribers received a retention call',
      df_unknowns['RetentionCalls'][df_unknowns['RetentionCalls'] > 0].count())
print('Subscribers received more than one retention call',
    df_unknowns['RetentionCalls'][df_unknowns['RetentionCalls'] > 1].count())


In [ ]:
# turn the two columns into binary columns, effectively clipping the ˜140 outliers

df_unknowns['RetentionCalls'] = df_unknowns['RetentionCalls'].clip(upper=1)

binary_columns.append('RetentionCalls')
continuous_columns.remove('RetentionCalls')
outlier_candidates.remove('RetentionCalls')

df_unknowns['ReferralsMadeBySubscriber'] = df_unknowns['ReferralsMadeBySubscriber'].clip(upper=1)

binary_columns.append('ReferralsMadeBySubscriber')
continuous_columns.remove('ReferralsMadeBySubscriber')
outlier_candidates.remove('ReferralsMadeBySubscriber')


There are some extreme outliers in a number columns. After visual inspection of the pair plots and the interquartile ranges above, and after confirmation that a small number of records are affected, let's remove them. 

We show outliers defined by 3* interquartile range on the log-transformed candidate columns. 

In [ ]:
from scipy import stats

print('Number of records outside of 3x interquartile range:\n')
log_data = np.log1p(df_unknowns[outlier_candidates])
for c in outlier_candidates: 
    print(c, log_data[log_data[c] > stats.iqr(log_data[c])*3].shape[0])

visually_inspected_outliers = {
    'UniqueSubs': 10, 
    'ActiveSubs': 5,
    'OffPeakCallsInOut': 800, 
    'PeakCallsInOut': 1000,
    'OverageMinutes': 1000,
    'RoamingCalls': 200,
    'DroppedCalls': 100,
    'BlockedCalls': 100,
    'ReceivedCalls': 1500,
    'UnansweredCalls': 500,
    'CustomerCareCalls': 100,
    'ReceivedCalls': 1800,
    'OutboundCalls': 500,
    'InboundCalls': 250,    
    'DroppedBlockedCalls': 250,
    'AdjustmentsToCreditRating': 10, 
    'TotalRecurringCharge': 200, 
    'MonthlyMinutes': 4500,
    'MonthlyRevenue': 600
}

# Since much of EDA is iterative work, we might have looked at outliers that we later decided
# to drop from the data altogether. Filter the list of outliers inspected by our drop candidates
for drop_item in drop_candidates: 
    if drop_item in visually_inspected_outliers:
        visually_inspected_outliers.pop(drop_item)

print('\nOutliers to be removed after visual inspection:\n')
for v_column, threshold in visually_inspected_outliers.items(): 
    print('Outliers to be removed from column {} : {}'.format(
        v_column, df_unknowns[df_unknowns[v_column] > threshold].shape[0]))


In [ ]:
for v_column, threshold in visually_inspected_outliers.items(): 
    df_outliers = df_unknowns[df_unknowns[v_column] <= threshold]
    
print('Number of records outside of 3x interquartile range, after removing extreme outliers:\n')
log_data = np.log1p(df_outliers[outlier_candidates])
for c in outlier_candidates: 
    print(c, log_data[log_data[c] > stats.iqr(log_data[c])*3].shape[0])

In [ ]:
fig = plt.figure(figsize = (20, 50))
j = 0
for c_column in continuous_columns:
    plt.subplot(20, 3, j+1)
    j += 1
    sns.distplot(df_outliers[c_column][df_outliers['Churn'] == 0], color='g', label = 'Remain')
    sns.distplot(df_outliers[c_column][df_outliers['Churn'] == 1], color='r', label = 'Churn')
    plt.legend(loc='best')
    
fig.suptitle('Churn Analysis Continous Variables', fontsize=20)
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

In [ ]:
log_transform_candidates = ['MonthlyMinutes', 'OverageMinutes', 
                            'RoamingCalls', 'DroppedCalls', 'BlockedCalls', 'UnansweredCalls',
                            'CustomerCareCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
                            'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls',
                            'CallForwardingCalls', 'UniqueSubs', 'ActiveSubs', 
                            'AdjustmentsToCreditRating'#, 'MonthlyRevenue', 'TotalRecurringCharge'
                           ]

log_transform_candidates = list(set(log_transform_candidates) - set(drop_candidates))
df_log1p = df_outliers

df_log1p.loc[:,log_transform_candidates] = np.log1p(df[log_transform_candidates])

In [ ]:
fig = plt.figure(figsize = (20, 50))
j = 0
for c_column in continuous_columns:
    plt.subplot(20, 3, j+1)
    j += 1
    sns.distplot(df_log1p[c_column][df_log1p['Churn'] == 0], color='g', label = 'Remain')
    sns.distplot(df_log1p[c_column][df_log1p['Churn'] == 1], color='r', label = 'Churn')
    plt.legend(loc='best')
    
fig.suptitle('Churn Analysis Continous Variables', fontsize=20)
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

In [ ]:
zero_inflated_candidates = ['' ]
continuous_columns

In [ ]:
#df = df[['Churn', 'MonthsInService', 'CurrentEquipmentDays']]

In [ ]:
df_bm = pd.get_dummies(df_log1p, columns=categorical_columns)

X = df_bm.drop(columns='Churn', axis=1)
y = df_bm['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)

## Resampling

The dataset is imbalanced, which will make typical classification algorithm perform very poorly on our target metric. Deploy resampling of the training set to balance the dataset. 

Try both under- and oversampling the training data and keep the more successful sampling strategy. 

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

print('Remain/Churn before resampling: ', Counter(y_train))

smote = SMOTE(random_state=random_state)
#rus = RandomUnderSampler(random_state=random_state)

X_train_sampled, y_train_sampled = smote.fit_resample(X_train, y_train)
#X_train_sampled, y_train_sampled = rus.fit_resample(X_train, y_train)

print('Remain/Churn after resampling: ', Counter(y_train_sampled))

Remain/Churn before resampling:  Counter({0: 27264, 1: 11011})
Remain/Churn after resampling:  Counter({1: 27264, 0: 27264})


In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train_sampled)

X_train = min_max_scaler.transform(X_train_sampled)
X_test = min_max_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_score

clfs = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=random_state), 
    'Logistic Regression': LogisticRegression(solver='lbfgs', random_state=random_state),
    'XGBoost': xgb.XGBClassifier(random_state=random_state)
}

for name, clf in clfs.items(): 
    #scores = cross_val_score(clf, X_train, y_train, cv=10, scoring=scorer)
    #print(name, 'Fbeta2', scores.mean())
    #scores = cross_val_score(clf, X_train, y_train, cv=10)
    #print(name, 'Accuracy', scores.mean())
    
    clf.fit(X_train_sampled, y_train_sampled)
    print(name, 'Test ', scorer(clf, X_test, y_test))
    

Random Forest Test  0.18002533019721367


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Logistic Regression Test  0.44328552803129073
XGBoost Test  0.2514061701039713


In [ ]:
from keras import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import tensorflow as tf

# https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
# http://proceedings.mlr.press/v54/eban17a/eban17a.pdf
# https://github.com/tensorflow/models/tree/master/research/global_objectives

def fbeta_metric_macro(y_true, y_pred, beta=2):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = (1 + beta ** 2) * p * r / ((beta ** 2) * p + r + K.epsilon())    
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def fbeta_loss_macro(y_true, y_pred, beta=2):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = (1 + beta ** 2) * p * r / ((beta ** 2) * p + r + K.epsilon())    
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

def make_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_initializer='random_normal', input_dim=X_train.shape[1]))
    model.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
    model.compile(optimizer ='adam',loss=fbeta_loss_macro, metrics=['accuracy', fbeta_metric_macro])
    return model

clf_keras = KerasClassifier(make_model) 
clf_keras.fit(X_train_sampled, y_train_sampled, batch_size=10, epochs=10)
print('Neural Net: ', scorer(clf_keras, X_test, y_test))

Epoch 1/10
54528/54528 [==============================] - 6s 108us/step - loss: 0.1892 - acc: 0.5000 - fbeta_metric_macro: 0.8111
Epoch 2/10
54528/54528 [==============================] - 5s 93us/step - loss: 0.1880 - acc: 0.5000 - fbeta_metric_macro: 0.8120
Epoch 3/10
54528/54528 [==============================] - 5s 90us/step - loss: 0.1888 - acc: 0.5000 - fbeta_metric_macro: 0.8112
Epoch 4/10
54528/54528 [==============================] - 5s 90us/step - loss: 0.1883 - acc: 0.5000 - fbeta_metric_macro: 0.8117
Epoch 5/10
54528/54528 [==============================] - 5s 90us/step - loss: 0.1884 - acc: 0.5000 - fbeta_metric_macro: 0.8116
Epoch 6/10
54528/54528 [==============================] - 5s 90us/step - loss: 0.1874 - acc: 0.5000 - fbeta_metric_macro: 0.8126
Epoch 7/10
54528/54528 [==============================] - 5s 90us/step - loss: 0.1884 - acc: 0.5000 - fbeta_metric_macro: 0.8116
Epoch 8/10
54528/54528 [==============================] - 5s 91us/step - loss: 0.1881 - acc: 0.5

In [ ]:
from sklearn.svm import SVC

clf_svc = SVC(random_state=random_state, gamma='auto')
clf_svc.fit(X_train_sampled, y_train_sampled)
print('SVC: ', scorer(clf_svc, X_test, y_test))